# Query Preprocsesing

In [1]:
pip install nltk

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/opt/bwhpc/common/jupyter/tensorflow/2022-03-30/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Loading necessary libraries
import pandas as pd
import nltk
from nltk.stem.porter import PorterStemmer

In [3]:
# Loading dataset 
queries = pd.read_json('data/train_topics_meta.jsonl', orient='columns')
# drop URL column
queries = queries.drop(columns='URL')

queries.head()

,id,keywords,title,rel_docs
0,84,"['cultivated', 'agricultural', 'maize', 'corn'...",Agriculture,"[572, 627, 678, 903, 1193, 1542, 1634, 3751, 3..."
1,111,"['reptile', 'lizard', 'salamander', 'fossil', ...",Amphibians and Reptiles,"[621, 809, 1380, 6641, 8311, 8937, 13134, 1446..."
2,265,"['astronomer', 'astronomy', 'astrophysicist', ...",Astronomy,"[39, 308, 580, 664, 736, 748, 791, 798, 799, 1..."
3,323,"['aviation', 'airfield', 'airport', 'aerospace...",Aviation,"[849, 852, 1293, 1902, 1942, 2039, 2075, 2082,..."
4,396,"['actor', 'cast', 'screenwriter', 'filmmaker',...",Biography/WikiProject Actors and Filmmakers,"[344, 676, 808, 872, 1247, 1806, 1828, 2083, 2..."


In [4]:
queries.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46 entries, 0 to 45
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46 non-null     int64 
 1   keywords  46 non-null     object
 2   title     46 non-null     object
 3   rel_docs  46 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.8+ KB


In [5]:
# convert string representatin of lists back to lists
queries['keywords'] = queries['keywords'].str.strip('][').str.replace("'", "").str.split(', ')
queries['rel_docs'] = queries['rel_docs'].str.strip('][').str.split(', ')
queries['rel_docs'] = queries['rel_docs'].apply(lambda l: [int(s) for s in l])

queries.head()

,id,keywords,title,rel_docs
0,84,"[cultivated, agricultural, maize, corn, fruit,...",Agriculture,"[572, 627, 678, 903, 1193, 1542, 1634, 3751, 3..."
1,111,"[reptile, lizard, salamander, fossil, frog, pr...",Amphibians and Reptiles,"[621, 809, 1380, 6641, 8311, 8937, 13134, 1446..."
2,265,"[astronomer, astronomy, astrophysicist, mathem...",Astronomy,"[39, 308, 580, 664, 736, 748, 791, 798, 799, 1..."
3,323,"[aviation, airfield, airport, aerospace, aircr...",Aviation,"[849, 852, 1293, 1902, 1942, 2039, 2075, 2082,..."
4,396,"[actor, cast, screenwriter, filmmaker, film, a...",Biography/WikiProject Actors and Filmmakers,"[344, 676, 808, 872, 1247, 1806, 1828, 2083, 2..."


In [6]:
# ========= Lower case whole column =========
queries['keywords'] = queries['keywords'].apply(lambda l: [s.lower() for s in l])
queries['keywords'].head()

0    [cultivated, agricultural, maize, corn, fruit,...
1    [reptile, lizard, salamander, fossil, frog, pr...
2    [astronomer, astronomy, astrophysicist, mathem...
3    [aviation, airfield, airport, aerospace, aircr...
4    [actor, cast, screenwriter, filmmaker, film, a...
Name: keywords, dtype: object

In [7]:
# ========= Removing special characters =========
import re

def remove_spec_char(plain_text):
    plain_text = re.sub('[^a-zA-Z0-9]', ' ', plain_text)
    plain_text = re.sub('\s+', ' ', plain_text)
    return plain_text

queries['keywords'] = queries['keywords'].apply(lambda l : [remove_spec_char(s) for s in l])

queries['keywords'].head()

0    [cultivated, agricultural, maize, corn, fruit,...
1    [reptile, lizard, salamander, fossil, frog, pr...
2    [astronomer, astronomy, astrophysicist, mathem...
3    [aviation, airfield, airport, aerospace, aircr...
4    [actor, cast, screenwriter, filmmaker, film, a...
Name: keywords, dtype: object

In [8]:
ps = PorterStemmer()
def stem_words(plain_text):
    return ' '.join([ps.stem(word) for word in plain_text.split()])

queries['keywords'] = queries['keywords'].apply(lambda l : [stem_words(s) for s in l])

In [9]:
# ========= Lemmatization =========
#from nltk import pos_tag
#from nltk.corpus import wordnet
#nltk.download('wordnet')
#nltk.download('omw-1.4')
#nltk.download('averaged_perceptron_tagger')
#from nltk.stem import WordNetLemmatizer

#lemmatizer = WordNetLemmatizer()
#wordnet_map = {'N':wordnet.NOUN, 'V':wordnet.VERB, 'J':wordnet.ADJ, 'R':wordnet.ADV}

#def lemmatize_word(plain_text):
#    # Finind pos tags
#    pos_text = pos_tag(plain_text.split())
#    return ' '.join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_text])

#queries['keywords'] = queries['keywords'].apply(lambda l : [lemmatize_word(s) for s in l])

#queries['keywords'].head()

In [10]:
queries.head()

,id,keywords,title,rel_docs
0,84,"[cultiv, agricultur, maiz, corn, fruit, wheat,...",Agriculture,"[572, 627, 678, 903, 1193, 1542, 1634, 3751, 3..."
1,111,"[reptil, lizard, salamand, fossil, frog, prehi...",Amphibians and Reptiles,"[621, 809, 1380, 6641, 8311, 8937, 13134, 1446..."
2,265,"[astronom, astronomi, astrophysicist, mathemat...",Astronomy,"[39, 308, 580, 664, 736, 748, 791, 798, 799, 1..."
3,323,"[aviat, airfield, airport, aerospac, aircraft,...",Aviation,"[849, 852, 1293, 1902, 1942, 2039, 2075, 2082,..."
4,396,"[actor, cast, screenwrit, filmmak, film, actre...",Biography/WikiProject Actors and Filmmakers,"[344, 676, 808, 872, 1247, 1806, 1828, 2083, 2..."


In [11]:
# save file
queries.to_csv('data/preprocessed_query_data.csv', index=False)